In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import Counter
from random import Random
from typing import Any, Iterator, Literal

from datasets import ClassLabel, Dataset, Value, load_dataset

from elk.promptsource import DatasetTemplates
from elk.utils import (
    assert_type,
    infer_label_column,
    select_split,
)
from elk.extraction.balanced_sampler import BalancedSampler, FewShotSampler


In [1]:
from elk.extraction import load_prompts, extract, Extract

In [6]:
data = extract(Extract("EleutherAI/pythia-1b", ("adzcai/genealogy_synthetic_v3",), use_encoder_states=True, token_loc="first"), num_gpus=1)

Using 1 of 4 GPUs: [1]
adzcai/genealogy_synthetic_v3 using 'train' for training and 'test' for validation
(1000, 1000) SplitInfo(name='train', num_bytes=683406, num_examples=2816, shard_lengths=None, dataset_name='parquet')


In [5]:
from datasets import DownloadMode

In [7]:
train_b = data['train']

In [8]:
train_b.download_and_prepare(download_mode=DownloadMode.FORCE_REDOWNLOAD)

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

GENERATING EXMAPLES
STARTING EXTRACTION WORKER
Model has language model head, will store predictions.
RUNNING CODE


Found cached dataset parquet (/home/ec2-user/.cache/huggingface/datasets/adzcai___parquet/adzcai--genealogy_synthetic_v3-5d84a0041cf3f45e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/ec2-user/.cache/huggingface/datasets/adzcai___parquet/adzcai--genealogy_synthetic_v3-5d84a0041cf3f45e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-1cef9e4ac338ad31.arrow
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


1
Using 1 variants of each prompt
No label column found, not balancing


In [5]:
train_b.as_dataset(split="train")

#     ds[split] = builder.as_dataset(split=split)

# dataset_dict = DatasetDict(ds)
# return DatasetDictWithName(
#     name=cfg.datasets[0],
#     dataset=dataset_dict,
# )

Dataset({
    features: ['hidden_0', 'hidden_1', 'hidden_2', 'hidden_3', 'hidden_4', 'hidden_5', 'hidden_6', 'hidden_7', 'hidden_8', 'hidden_9', 'hidden_10', 'hidden_11', 'hidden_12', 'hidden_13', 'hidden_14', 'hidden_15', 'hidden_16', 'variant_ids', 'label', 'text_questions', 'model_logits'],
    num_rows: 250
})

In [6]:
data['train']

In [2]:
df = list(load_prompts("adzcai/genealogy_synthetic_v3"))

RUNNING CODE


Found cached dataset parquet (/home/ec2-user/.cache/huggingface/datasets/adzcai___parquet/adzcai--genealogy_synthetic_v3-5d84a0041cf3f45e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/ec2-user/.cache/huggingface/datasets/adzcai___parquet/adzcai--genealogy_synthetic_v3-5d84a0041cf3f45e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-1cef9e4ac338ad31.arrow


1
Using 1 variants of each prompt
No label column found, not balancing


In [3]:
df

[{'label': 0,
  'prompts': [[{'answer': 'Josiah',
     'question': 'Samuel and Harrison are the sons of Josiah. Ryan and Benjamin are the sons of Samuel. Lucas and Ryker are the sons of Harrison. Theodore and Joshua are the sons of Lucas. Who is the father of Samuel?'},
    {'answer': 'Benjamin',
     'question': 'Samuel and Harrison are the sons of Josiah. Ryan and Benjamin are the sons of Samuel. Lucas and Ryker are the sons of Harrison. Theodore and Joshua are the sons of Lucas. Who is the father of Samuel?'},
    {'answer': 'Samuel',
     'question': 'Samuel and Harrison are the sons of Josiah. Ryan and Benjamin are the sons of Samuel. Lucas and Ryker are the sons of Harrison. Theodore and Joshua are the sons of Lucas. Who is the father of Samuel?'},
    {'answer': 'Ryker',
     'question': 'Samuel and Harrison are the sons of Josiah. Ryan and Benjamin are the sons of Samuel. Lucas and Ryker are the sons of Harrison. Theodore and Joshua are the sons of Lucas. Who is the father of S

In [ ]:
def foo():
    ds_string: str = "adzcai/genealogy_synthetic_v3"

    binarize: bool = False
    num_shots: int = 0
    num_variants: int = -1
    seed: int = 42
    split_type: Literal["train", "val"] = "train"
    template_path: str | None = None
    rank: int = 0
    world_size: int = 1

    """Load a dataset full of prompts generated from the specified dataset.

    Args:
        ds_string: Name of HF dataset to use, e.g. `"super_glue:boolq"` or `"imdb"`.
        binarize: Whether to binarize the dataset labels for multi-class datasets.
        num_shots: The number of examples to use in few-shot prompts. If zero, prompts
            are zero-shot.
        seed: The seed to use for prompt randomization.
        split_type: Whether to use the train or val split of the dataset.
        template_path: Path to feed into `DatasetTemplates` for loading templates.
        rank: The rank of the current process. Defaults to 0.
        world_size: The number of processes. Defaults to 1.

    Returns:
        An iterable of prompt dictionaries.
    """
    print("RUNNING CODE")
    ds_name, _, config_name = ds_string.partition(":")

    ds_dict = assert_type(dict, load_dataset(ds_name, config_name or None))
    split_name = select_split(ds_dict, split_type)

    ds = assert_type(Dataset, ds_dict[split_name].shuffle(seed=seed))
    if world_size > 1:
        ds = ds.shard(world_size, rank)

    if template_path is None:
        prompter = DatasetTemplates(ds_name, config_name)
    else:
        prompter = DatasetTemplates(template_path)

    # If the prompt template says to binarize, we should
    binarize = binarize or prompter.binarize
    prompter.drop_non_mc_templates()

    num_templates = len(prompter.templates)
    print(num_templates)
    num_variants = (
        num_templates if num_variants == -1 else min(num_variants, num_templates)
    )
    assert num_variants > 0
    if rank == 0:
        print(f"Using {num_variants} variants of each prompt")

    label_column = prompter.label_column or infer_label_column(ds.features)

    label_feature = ds.features[label_column]
    if isinstance(label_feature, ClassLabel):
        label_choices = [label_feature.str2int(label) for label in label_feature.names]
    elif isinstance(label_feature, Value) and label_feature.dtype == "bool":
        label_choices = [False, True]
    else:
        # Which classes are actually present in this split of the dataset?
        # This is shockingly fast since it uses an optimized Apache Arrow primitive.
        label_choices = sorted(ds.unique(label_column))
        if rank == 0:
            print(f"Using the following pseudo-labels: {label_choices}")

    rng = Random(seed)
    if num_shots > 0:
        train_name = select_split(ds_dict, "train")
        fewshot = FewShotSampler(
            ds_dict[train_name].shuffle(seed=seed),  # TODO: not iterator
            num_shots=num_shots,
            rng=rng,
        )
        fewshot_iter = iter(fewshot)
    else:
        fewshot_iter = None

    # if label_column in ds.features:
    #     ds = BalancedSampler(
    #         ds.to_iterable_dataset(),
    #         set(label_choices),
    #         label_col=label_column,
    #     )
    # else:
    if rank == 0:
        print("No label column found, not balancing")
    ds = ds.to_iterable_dataset()

    print(ds)

    for example in ds:
        yield _convert_to_prompts(
            example,
            binarize=binarize,
            label_column=label_column,
            label_choices=label_choices,  # type: ignore[arg-type]
            num_variants=num_variants,
            prompter=prompter,
            rng=rng,
            fewshot_iter=fewshot_iter,
        )

In [ ]:
list(foo())

In [ ]:
list(prompts)

In [ ]:
data = extract_hiddens(Extract("EleutherAI/pythia-1b", ["adzcai/genealogy_synthetic_v3"]))
data = list(data)

In [ ]:
df = load_dataset("adzcai/genealogy_synthetic_v3", cache_dir="/tmp")

In [ ]:
data

In [ ]:
df['train']['label']

In [ ]:
df['train'].features

In [ ]:
df

In [ ]:
df['train'].features